<a href="https://colab.research.google.com/github/AdityaTheDev/Pytorch-learnings/blob/main/optuna_basics_on_classical_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 8.4 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-06-09 04:36:29,779] A new study created in memory with name: no-name-7dc2a1fe-486d-4ebc-8507-5663f402abec
[I 2025-06-09 04:36:31,940] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 112, 'max_depth': 8}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-06-09 04:36:34,823] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 186, 'max_depth': 17}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-06-09 04:36:36,085] Trial 2 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 107, 'max_depth': 3}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-06-09 04:36:38,522] Trial 3 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 177, 'max_depth': 20}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-06-09 04:36:40,409] Trial 4 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 152, 'max_depth': 11}. Best is trial 1 with value: 0.770949

In [7]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 119, 'max_depth': 18}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


## Samplers in Optuna

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-06-09 04:37:19,510] A new study created in memory with name: no-name-e2713ee7-cf45-451b-b70d-fad47cc2c873
[I 2025-06-09 04:37:20,048] Trial 0 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 97, 'max_depth': 17}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-06-09 04:37:20,970] Trial 1 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 183, 'max_depth': 5}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-06-09 04:37:21,510] Trial 2 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-06-09 04:37:22,067] Trial 3 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 79, 'max_depth': 18}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-06-09 04:37:23,166] Trial 4 finished with value: 0.7783985102420856 and parameters: {'n_estimators': 131, 'max_depth': 19}. Best is trial 4 with value: 0.7783985

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-06-09 04:37:53,099] A new study created in memory with name: no-name-02f451cf-f435-4cc6-a784-f8a91794c880
[I 2025-06-09 04:37:53,611] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-06-09 04:37:54,418] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-06-09 04:37:54,706] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-06-09 04:37:55,266] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-06-09 04:37:55,808] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [17]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [18]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [19]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [20]:
# 1. Optimization History
plot_optimization_history(study).show()

In [21]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [22]:
# 3. Slice Plot
plot_slice(study).show()

In [23]:
# 4. Contour Plot
plot_contour(study).show()

In [24]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [25]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [26]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [27]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-06-09 04:39:12,297] A new study created in memory with name: no-name-b82eebf8-a96c-4441-bb17-aa1b99c78b96
[I 2025-06-09 04:39:12,333] Trial 0 finished with value: 0.7579143389199254 and parameters: {'classifier': 'SVM', 'C': 0.911233409060198, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.7579143389199254.
[I 2025-06-09 04:39:13,457] Trial 1 finished with value: 0.7728119180633147 and parameters: {'classifier': 'RandomForest', 'n_estimators': 268, 'max_depth': 14, 'min_samples_split': 2, 'min_samples_leaf': 8, 'bootstrap': False}. Best is trial 1 with value: 0.7728119180633147.
[I 2025-06-09 04:39:13,496] Trial 2 finished with value: 0.7765363128491619 and parameters: {'classifier': 'SVM', 'C': 0.19465032963922538, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 2 with value: 0.7765363128491619.
[I 2025-06-09 04:39:15,179] Trial 3 finished with value: 0.7616387337057727 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 143, 'learning

In [28]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.11825325108220569, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [29]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.757914,2025-06-09 04:39:12.298540,2025-06-09 04:39:12.333239,0 days 00:00:00.034699,0.911233,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.772812,2025-06-09 04:39:12.334363,2025-06-09 04:39:13.456932,0 days 00:00:01.122569,NaN,False,RandomForest,NaN,NaN,NaN,14.0,8.0,2.0,268.0,COMPLETE
2,2,0.776536,2025-06-09 04:39:13.458153,2025-06-09 04:39:13.495956,0 days 00:00:00.037803,0.194650,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.761639,2025-06-09 04:39:13.497032,2025-06-09 04:39:15.178845,0 days 00:00:01.681813,NaN,NaN,GradientBoosting,NaN,NaN,0.076213,13.0,8.0,7.0,143.0,COMPLETE
4,4,0.761639,2025-06-09 04:39:15.180148,2025-06-09 04:39:15.833776,0 days 00:00:00.653628,NaN,True,RandomForest,NaN,NaN,NaN,15.0,10.0,7.0,131.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.785847,2025-06-09 04:39:45.798566,2025-06-09 04:39:45.834648,0 days 00:00:00.036082,0.259778,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.715084,2025-06-09 04:39:45.835761,2025-06-09 04:39:45.870195,0 days 00:00:00.034434,0.186653,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.783985,2025-06-09 04:39:45.871298,2025-06-09 04:39:45.903271,0 days 00:00:00.031973,0.346041,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.733706,2025-06-09 04:39:45.904337,2025-06-09 04:39:48.923561,0 days 00:00:03.019224,NaN,NaN,GradientBoosting,NaN,NaN,0.147630,15.0,9.0,3.0,206.0,COMPLETE


In [30]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,79
RandomForest,11
GradientBoosting,10


In [31]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.748603
RandomForest,0.761977
SVM,0.774415


In [32]:
# 1. Optimization History
plot_optimization_history(study).show()

In [33]:
# 3. Slice Plot
plot_slice(study).show()

In [34]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [37]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-06-09 04:41:08,336] A new study created in memory with name: no-name-af05042d-f011-4b6c-88ea-87fe173eb8d4


[0]	train-mlogloss:0.89210	eval-mlogloss:0.89681
[1]	train-mlogloss:0.69005	eval-mlogloss:0.68116
[2]	train-mlogloss:0.55007	eval-mlogloss:0.53699
[3]	train-mlogloss:0.44570	eval-mlogloss:0.42894
[4]	train-mlogloss:0.36628	eval-mlogloss:0.34403
[5]	train-mlogloss:0.30511	eval-mlogloss:0.27711
[6]	train-mlogloss:0.25889	eval-mlogloss:0.22937
[7]	train-mlogloss:0.22267	eval-mlogloss:0.18995
[8]	train-mlogloss:0.19471	eval-mlogloss:0.15973
[9]	train-mlogloss:0.17339	eval-mlogloss:0.13733
[10]	train-mlogloss:0.15759	eval-mlogloss:0.11875
[11]	train-mlogloss:0.14673	eval-mlogloss:0.10342
[12]	train-mlogloss:0.14042	eval-mlogloss:0.09975
[13]	train-mlogloss:0.13563	eval-mlogloss:0.09115
[14]	train-mlogloss:0.13286	eval-mlogloss:0.08673
[15]	train-mlogloss:0.13023	eval-mlogloss:0.08180
[16]	train-mlogloss:0.12765	eval-mlogloss:0.07869
[17]	train-mlogloss:0.12502	eval-mlogloss:0.07588
[18]	train-mlogloss:0.12296	eval-mlogloss:0.07298
[19]	train-mlogloss:0.12213	eval-mlogloss:0.07149
[20]	train

[I 2025-06-09 04:41:13,349] Trial 0 finished with value: 1.0 and parameters: {'lambda': 0.0003102242994822896, 'alpha': 6.787002837447456e-08, 'eta': 0.22521046228791475, 'gamma': 0.0001833429197747007, 'max_depth': 5, 'min_child_weight': 5, 'subsample': 0.9757825551894583, 'colsample_bytree': 0.5084446744469432}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.99023	eval-mlogloss:0.99125
[1]	train-mlogloss:0.85232	eval-mlogloss:0.84565
[2]	train-mlogloss:0.74152	eval-mlogloss:0.72815
[3]	train-mlogloss:0.64776	eval-mlogloss:0.63050
[4]	train-mlogloss:0.57134	eval-mlogloss:0.54914
[5]	train-mlogloss:0.50405	eval-mlogloss:0.47501
[6]	train-mlogloss:0.45027	eval-mlogloss:0.41979
[7]	train-mlogloss:0.40596	eval-mlogloss:0.37159
[8]	train-mlogloss:0.36658	eval-mlogloss:0.32815
[9]	train-mlogloss:0.33189	eval-mlogloss:0.29083
[10]	train-mlogloss:0.30323	eval-mlogloss:0.25853
[11]	train-mlogloss:0.28065	eval-mlogloss:0.23185
[12]	train-mlogloss:0.26896	eval-mlogloss:0.21891
[13]	train-mlogloss:0.25004	eval-mlogloss:0.19626
[14]	train-mlogloss:0.24043	eval-mlogloss:0.18559
[15]	train-mlogloss:0.22783	eval-mlogloss:0.17158
[16]	train-mlogloss:0.21864	eval-mlogloss:0.16190
[17]	train-mlogloss:0.20981	eval-mlogloss:0.15230
[18]	train-mlogloss:0.20106	eval-mlogloss:0.14230
[19]	train-mlogloss:0.19984	eval-mlogloss:0.14060
[20]	train

[I 2025-06-09 04:41:25,463] Trial 1 finished with value: 1.0 and parameters: {'lambda': 0.048559771227142993, 'alpha': 0.23318260150746845, 'eta': 0.13183005191143676, 'gamma': 7.740582672930084e-06, 'max_depth': 8, 'min_child_weight': 6, 'subsample': 0.7215618616955517, 'colsample_bytree': 0.6991165225990208}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:1.04048	eval-mlogloss:1.04590
[1]	train-mlogloss:0.98814	eval-mlogloss:0.98948
[2]	train-mlogloss:0.93245	eval-mlogloss:0.93261


[I 2025-06-09 04:41:36,122] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:1.07428	eval-mlogloss:1.07423
[1]	train-mlogloss:1.04089	eval-mlogloss:1.03944
[2]	train-mlogloss:1.00874	eval-mlogloss:1.00573
[3]	train-mlogloss:0.97780	eval-mlogloss:0.97403
[4]	train-mlogloss:0.94848	eval-mlogloss:0.94419
[5]	train-mlogloss:0.92029	eval-mlogloss:0.91536
[6]	train-mlogloss:0.89335	eval-mlogloss:0.88767
[7]	train-mlogloss:0.86762	eval-mlogloss:0.86154
[8]	train-mlogloss:0.84314	eval-mlogloss:0.83620


[I 2025-06-09 04:41:36,253] Trial 49 pruned. Trial was pruned at iteration 8.


Best trial: {'lambda': 0.0003102242994822896, 'alpha': 6.787002837447456e-08, 'eta': 0.22521046228791475, 'gamma': 0.0001833429197747007, 'max_depth': 5, 'min_child_weight': 5, 'subsample': 0.9757825551894583, 'colsample_bytree': 0.5084446744469432}
Best accuracy: 1.0


In [36]:
! pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 4.2 MB/s eta 0:00:00


In [38]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()